In [4]:
import sys
import os

this_path = os.getcwd()
path = '\\'.join(this_path.split('\\')[:-1])+'\\'
if path not in sys.path:
    sys.path.insert(0, path)
    print(path+" added to python path")
else:
    print(path+" already in python path")
    
import pandas as pd
import datetime as dt
import pickle
import math
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2


C:\Users\mgutierrez\root\Projects\SDC\py\kaissandra\ already in python path
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Find candidate

In [18]:
from kaissandra.local_config import local_vars

results_filename = local_vars.results_directory
IDresults = 'RRNN01010k5K5AC'#'RRNN00000K5ACNR'
results_table_filename = results_filename+IDresults+'/performance.csv'
TR = pd.read_csv(results_table_filename, sep='\t')

In [19]:
def extract_result(dict_inputs, list_kpis):
    """ Extract single results """
    # init idxs to Trues
    idxs = TR['epoch']>=0
    for key,func in dict_inputs.items():
        idxs = idxs & (TR[key].apply(func))
    dict_out = {}
    for kpi in list_kpis:
        maxidx = TR[kpi][idxs].idxmax()
        dict_out[kpi] = TR.iloc[maxidx]
        print(kpi)
        print(dict_out[kpi].to_string())

In [20]:
func1 = lambda x:x=='mean'
func2 = lambda x:x>=60
func3 = lambda x:x==12
func_mc = lambda x:x==0.7
func_md = lambda x:x==0.6
extract_result({'thr_mc':func_mc,'thr_md':func_md,'t_index':func1}, ['eROI2'])#,

eROI2
epoch                9
t_index           mean
thr_mc             0.7
thr_md             0.6
AD             57.5772
ADA            44.4644
GSP            59.3803
NSP            54.5915
NO                1775
NZ                8189
NZA              10604
RD                4715
NSP.5          58.4225
NSP1           57.5775
NSP1.5         56.7324
NSP2            55.831
NSP3           53.5211
NSP4            51.493
NSP5           48.9014
SI.5           7.09695
SI1            6.38489
SI1.5          5.67282
SI2            4.91328
SI3            2.96695
SI4            1.25799
SI5           -0.92569
SI             -19.118
eGROI          77.4995
eROI.5         68.6245
eROI1          59.7495
eROI1.5        50.8745
eROI2          41.9995
eROI3          24.2495
eROI4          6.49945
eROI5         -11.2505
eROI           34.1878
mSpread      0.0244009
pNZ           0.919492
pNZA           1.19066
tGROI          246.823
tROI          -31.4662
eRl1           34.1878
eRl2                 0
eGl1 

## Get Positions Candidate

In [11]:
from kaissandra.automate import automate_Kfold

rootname_config = 'RNN01010'
extR = 'AC'
sufix=''
K = 5
k_end = 5
resolution = 10
k_init = 2
outputGain = 1
movingWindow = 500
nEventsPerStat = 5000
results_from = 'COMB'
feats_from_bids = False
save_journal = False
t_indexes = [0,1,2,3]
thresholds_mc = [.7]#[int(np.round((.5+i/resolution)*100))/100 for i in range(int(resolution/2))]
thresholds_md = [.6]#[int(np.round((.5+i/resolution)*100))/100 for i in range(int(resolution/2))]
IDrs=['R'+rootname_config+'k'+str(fold_idx+1)+'K'+str(K)+extR+sufix for fold_idx in range(k_end)]
IDr_merged=rootname_config+'k'+str(k_init+1)+'-'+str(k_end)+'K'+str(K)+extR+sufix

In [12]:
entries = {'thresholds_mc':thresholds_mc,'thresholds_md':thresholds_md,'resolution':resolution,'results_from':results_from,'startFrom':5,
           'endAt':-1,'save_journal':save_journal,'feats_from_bids':feats_from_bids,'size_output_layer':8,'n_bits_outputs':[1,2,5],
           'combine_ts':{'if_combine':True,'params_combine':[{'alg':'mean'}]},'outputGain':outputGain,'movingWindow':movingWindow,
           'nEventsPerStat':nEventsPerStat,'t_indexes':t_indexes}
automate_Kfold(rootname_config, entries=entries, K=K, tAt='Te', IDrs=IDrs, build_IDrs=False,
            its=1, sufix=sufix, IDr_merged=IDr_merged,k_init=k_init, k_end=k_end,if_merge_results=False)

k 3 of 5
IDresults:
RRNN01010k3K5AC
WARNING! Config values have been overwritten with entries values
Config file C:/Users/mgutierrez/root/Projects/SDC/py/RNN/config/CRNN01010k3K5.config exists. Loaded from disk
RRNN01010k3K5AC
Getting dataset edges...
C:/Users/mgutierrez/root/Projects/SDC/py/RNN/IO/DTA_RRNN01010k3K5AC.p
Edges:
[datetime.date(2016, 8, 2), datetime.date(2017, 3, 6), datetime.date(2017, 9, 27), datetime.date(2018, 4, 10)]
C:/Users/mgutierrez/root/Projects/SDC/py/RNN/IO/IOKFWRNN01010k3K5AC.hdf5
C:/Users/mgutierrez/root/Projects/SDC/py/RNN/IO/IOKFRRNN01010k3K5AC.hdf5
C:/Users/mgutierrez/root/Projects/SDC/py/RNN/IO/DTA_RRNN01010k3K5AC.p
Samples for fitting: 3051829
Samples for cross-validation: 752867
Total samples: 3804696
totalSampsPerLevel not in IO :(
DONE building IO
Iteration 0 of 0
INFO:tensorflow:Restoring parameters from C:/Users/mgutierrez/root/Projects/SDC/py/RNN/weights/WRNN01010k3K5A/000005/000005
Parameters loaded. Epoch 5
Epoch 5 of 14. Getting output...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 736/736 [01:02<00:00, 11.70it/s]


J_test = 1.556141
Epoch 5, J_train = 1.6490453079910292, J_test = 1.556140737041183
Getting extended results...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 62/62 [00:00<00:00, 1480.03it/s]


Epoch = 5. Time index = 0. Threshold MC = 0.7. Threshold MD = 0.6
RD = 30 NZ = 46 NZA = 63 pNZ = 0.006% pNZA = 0.008% AD = 65.22% ADA = 47.62% NO = 60 GSP = 58.33% NSP = 53.33%
SI2 = 1.03 SI = -3.55 eGROI = 2.38% eROI = 0.95% eROI2 = 1.18% eROI3 = 0.58% mSpread = 0.0237%

Getting extended results...


...

Epoch = 14. Time index = mean. Threshold MC = 0.7. Threshold MD = 0.6
RD = 10904 NZ = 19709 NZA = 25548 pNZ = 2.213% pNZA = 2.869% AD = 55.32% ADA = 42.68% NO = 3939 GSP = 54.89% NSP = 50.57%
SI2 = 1.48 SI = -31.01 eGROI = 98.21% eROI = 5.67% eROI2 = 19.43% eROI3 = -19.96% mSpread = 0.0235%

Getting extended results...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25547/25547 [00:05<00:00, 4608.50it/s]


Epoch = 14. Time index = mean. Threshold MC = 0.7. Threshold MD = 0.6
RD = 10904 NZ = 19709 NZA = 25548 pNZ = 2.213% pNZA = 2.869% AD = 55.32% ADA = 42.68% NO = 3939 GSP = 54.89% NSP = 50.57%
SI2 = 1.48 SI = -31.01 eGROI = 98.21% eROI = 5.67% eROI2 = 19.43% eROI3 = -19.96% mSpread = 0.0235%

Getting extended results...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25547/25547 [00:05<00:00, 4748.61it/s]


Epoch = 14. Time index = mean. Threshold MC = 0.7. Threshold MD = 0.6
RD = 10904 NZ = 19709 NZA = 25548 pNZ = 2.213% pNZA = 2.869% AD = 55.32% ADA = 42.68% NO = 3939 GSP = 54.89% NSP = 50.57%
SI2 = 1.48 SI = -31.01 eGROI = 98.21% eROI = 5.67% eROI2 = 19.43% eROI3 = -19.96% mSpread = 0.0235%

Best eGROI = 182.74 t_index 1 thr_mc 0.5 thr_md 0.6 epoch 14 in 12364 GSP 56.94 ADA 38.43 AD 55.64
Best eROI = 31.47 t_index 3 thr_mc 0.9 thr_md 0.5 epoch 14 in 184 GSP 55.43 ADA 49.93 AD 55.90
Best eROI.5 = 120.92 t_index 1 thr_mc 0.5 thr_md 0.6 epoch 14 in 12364 GSP 56.94 ADA 38.43 AD 55.64
Best eROI1 = 76.50 t_index 3 thr_mc 0.5 thr_md 0.7 epoch 14 in 3365 GSP 60.77 ADA 42.78 AD 57.60
Best eROI1.5 = 59.68 t_index 3 thr_mc 0.5 thr_md 0.7 epoch 14 in 3365 GSP 60.77 ADA 42.78 AD 57.60
Best eROI2 = 42.85 t_index 3 thr_mc 0.5 thr_md 0.7 epoch 14 in 3365 GSP 60.77 ADA 42.78 AD 57.60
Best eROI3 = 31.30 t_index 3 thr_mc 0.9 thr_md 0.5 epoch 14 in 184 GSP 55.43 ADA 49.93 AD 55.90
Best eROI4 = 29.46 t_ind

In [ ]:
from kaissandra.automate import automate_Kfold

rootname_config = 'RNN01021'
extR = 'AL'
sufix='R20'
K = 5
k_end = 2
resolution = 20
k_init = 1
outputGain = 0.8
movingWindow = 200
nEventsPerStat = 2000
results_from = 'ASKS'
feats_from_bids = False
save_journal = True
thresholds_mc = [int(np.round((.5+i/resolution)*100))/100 for i in range(int(resolution/2))]
thresholds_md = [int(np.round((.5+i/resolution)*100))/100 for i in range(int(resolution/2))]
IDrs=['R'+rootname_config+'k'+str(fold_idx+1)+'K'+str(K)+extR+sufix for fold_idx in range(k_end)]
IDr_merged=rootname_config+'k'+str(k_init+1)+'-'+str(k_end)+'K'+str(K)+extR+sufix

In [ ]:
entries = {'thresholds_mc':thresholds_mc,'thresholds_md':thresholds_md,'resolution':resolution,'results_from':results_from,'startFrom':9,
           'endAt':9,'save_journal':save_journal,'feats_from_bids':feats_from_bids,'size_output_layer':8,'n_bits_outputs':[1,2,5],
           'combine_ts':{'if_combine':False,'params_combine':[{'alg':'mean'}]},'outputGain':outputGain,
           'movingWindow':movingWindow,'nEventsPerStat':nEventsPerStat}
automate_Kfold(rootname_config, entries=entries, K=K, tAt='Te', IDrs=IDrs, build_IDrs=False,
            its=1, sufix=sufix, IDr_merged=IDr_merged,k_init=k_init, k_end=k_end,if_merge_results=False)

In [ ]:
from kaissandra.results2 import merge_results
merge_results(IDrs, IDr_merged)

## Test unseen dataset

In [ ]:
from kaissandra.automate import automate_fixedEdges


rootname_config = 'RNN01010'
k = 1
outputGain = 1
movingWindow = 500
nEventsPerStat = 5000
results_from = 'ASKS'
feats_from_bids = False
save_journal = True
resolution = 10
thresholds_mc = [.6]#[int(np.round((.5+i/resolution)*100))/100 for i in range(int(resolution/2))]
thresholds_md = [.7]#[int(np.round((.5+i/resolution)*100))/100 for i in range(int(resolution/2))]
startFrom = 12
endAt = 12
sufix='testk'+str(k)+'E'+str(startFrom)+'TmeanC6D6'
t_indexes = [0,1,2,3]
if feats_from_bids:
    ext = 'B'
else:
    ext = 'A'
IDweights = 'W'+rootname_config+'k'+str(k)+'K5'+ext
first_day = '2018.11.12'
last_day = '2019.03.29'
list_index2sets = ['Cv']
edge_dates = []
combine_ts = {'if_combine':True,'params_combine':[{'alg':'mean'}]}

entries = {'thresholds_mc':thresholds_mc,'thresholds_md':thresholds_md,'resolution':resolution,'results_from':results_from,'startFrom':startFrom,
           'endAt':endAt,'save_journal':save_journal,'feats_from_bids':feats_from_bids,'size_output_layer':8,'n_bits_outputs':[1,2,5],
           'combine_ts':{'if_combine':True,'params_combine':[{'alg':'mean'}]},'outputGain':outputGain,'movingWindow':movingWindow,
           'nEventsPerStat':nEventsPerStat,'first_day':first_day,'last_day':last_day,'list_index2sets':list_index2sets,'edge_dates':edge_dates,
          'build_test_db':True,'save_stats':False,'t_indexes':t_indexes,'combine_ts':combine_ts}

automate_fixedEdges(rootname_config,entries=entries,its=1,tAt='Te',sufix=sufix,IDweights=IDweights)

In [ ]:
entries['feats_from_bids']

## Combine Models

In [21]:
import sys
import os

this_path = os.getcwd()
path = '\\'.join(this_path.split('\\')[:-1])+'\\'
if path not in sys.path:
    sys.path.insert(0, path)
    print(path+" added to python path")
else:
    print(path+" already in python path")
    
import pandas as pd
import datetime as dt
import pickle
import math
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2
#######################################
# For test dataset

from kaissandra.automate import combine_models
import numpy as np

rootname_config = 'RNN01010'
outputGain = 1
movingWindow = 500
nEventsPerStat = 5000
results_from = 'COMB'
feats_from_bids = False
save_journal = True
resolution = 20
thresholds_mc = [.7]#[int(np.round((.5+i/resolution)*100))/100 for i in range(int(resolution/2))]#
thresholds_md = [.6]#[int(np.round((.5+i/resolution)*100))/100 for i in range(int(resolution/2))]#
epochs = [12,14,13,11,9]#[9,11,6,8,6]
sufix='k1-5'+'E1214131109'#'k1k2E12E14'#
t_indexes = [0]
if feats_from_bids:
    ext = 'B'
else:
    ext = 'A'
first_day = '2018.11.12'#'2016.01.01'#
last_day = '2019.05.01'#'2018.11.09'#
list_index2sets = ['Cv']#['No','Cv','No']#['No','Cv']#
edge_dates = []#['2019.04.01']#['2017.9.27','2018.4.10']#['2018.4.10']#
combine_ts = {'if_combine':False,'params_combine':[{'alg':'mean'}]}

entries = {'thresholds_mc':thresholds_mc,'thresholds_md':thresholds_md,'resolution':resolution,'results_from':results_from,
           'save_journal':save_journal,'feats_from_bids':feats_from_bids,'size_output_layer':8,'n_bits_outputs':[1,2,5],
           'combine_ts':combine_ts,'outputGain':outputGain,'movingWindow':movingWindow,
          'nEventsPerStat':nEventsPerStat,'first_day':first_day,'last_day':last_day,'list_index2sets':list_index2sets,'edge_dates':edge_dates,
          'build_test_db':True,'save_stats':False,'t_indexes':t_indexes,'combine_ts':combine_ts}

model_names = ['WRNN01010k1K5A','WRNN01010k2K5A','WRNN01010k3K5A','WRNN01010k4K5A','WRNN01010k5K5A']#

tag_from_till = 'F181112T190426'#''F190401T190426'#F170927T180410'#'F180410T181109'#
combine_models(entries, model_names, epochs, rootname_config, sufix=sufix, melting_func='mean', tag_from_till=tag_from_till,log='')

C:\Users\mgutierrez\root\Projects\SDC\py\kaissandra\ already in python path
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
WARNING! Config values have been overwritten with entries values
Config file C:/Users/mgutierrez/root/Projects/SDC/py/RNN/config/CRNN01010CMF181112T190426.config exists. Loaded from disk
Getting dataset edges...
C:/Users/mgutierrez/root/Projects/SDC/py/RNN/IO/DTA_RRNN01010CMF181112T190426A.p
Edges:
[]
C:/Users/mgutierrez/root/Projects/SDC/py/RNN/IO/IOKFWRNN01010CMF181112T190426A.hdf5
C:/Users/mgutierrez/root/Projects/SDC/py/RNN/IO/IOKFRRNN01010CMF181112T190426A.hdf5
C:/Users/mgutierrez/root/Projects/SDC/py/RNN/IO/DTA_RRNN01010CMF181112T190426A.p
Samples for fitting: 0
Samples for cross-validation: 389033
Total samples: 389033
totalSampsPerLevel not in IO :(
DONE building IO
INFO:tensorflow:Restoring parameters from C:/Users/mgutierrez/root/Projects/SDC/py/RNN/weights/WRNN01010k1K5A/000012/000012
Parameters loaded. Epoch 12
E

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:33<00:00, 11.21it/s]


J_test = 13.284653
Total time for CV: 0.571993092695872 mins.

INFO:tensorflow:Restoring parameters from C:/Users/mgutierrez/root/Projects/SDC/py/RNN/weights/WRNN01010k2K5A/000014/000014
Parameters loaded. Epoch 14
Epoch 14 of 14. Getting output...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:31<00:00, 11.88it/s]


J_test = 12.323366
Total time for CV: 0.5412746071815491 mins.

INFO:tensorflow:Restoring parameters from C:/Users/mgutierrez/root/Projects/SDC/py/RNN/weights/WRNN01010k3K5A/000013/000013
Parameters loaded. Epoch 13
Epoch 13 of 13. Getting output...


...

J_test = 12.641410
Total time for CV: 0.5333674311637878 mins.

INFO:tensorflow:Restoring parameters from C:/Users/mgutierrez/root/Projects/SDC/py/RNN/weights/WRNN01010k5K5A/000009/000009
Parameters loaded. Epoch 9
Epoch 9 of 9. Getting output...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:30<00:00, 12.49it/s]


J_test = 12.202218
Total time for CV: 0.5156976898511251 mins.

Epoch 0, J_train = 0.0, J_test = 12.202218072037947
Getting extended results...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1160/1160 [00:00<00:00, 3294.52it/s]


Epoch = 0. Time index = 0. Threshold MC = 0.7. Threshold MD = 0.6
RD = 574 NZ = 878 NZA = 1161 pNZ = 0.226% pNZA = 0.298% AD = 65.38% ADA = 49.44% NO = 330 GSP = 61.21% NSP = 53.94%
SI2 = 1.54 SI = -8.34 eGROI = 12.83% eROI = 3.86% eROI2 = 6.23% eROI3 = 2.93% mSpread = 0.0272%

Best eGROI = 12.83 t_index 0 thr_mc 0.7 thr_md 0.6 epoch 0 in 330 GSP 61.21 ADA 49.44 AD 65.38
Best eROI = 3.86 t_index 0 thr_mc 0.7 thr_md 0.6 epoch 0 in 330 GSP 61.21 ADA 49.44 AD 65.38
Best eROI.5 = 11.18 t_index 0 thr_mc 0.7 thr_md 0.6 epoch 0 in 330 GSP 61.21 ADA 49.44 AD 65.38
Best eROI1 = 9.53 t_index 0 thr_mc 0.7 thr_md 0.6 epoch 0 in 330 GSP 61.21 ADA 49.44 AD 65.38
Best eROI1.5 = 7.88 t_index 0 thr_mc 0.7 thr_md 0.6 epoch 0 in 330 GSP 61.21 ADA 49.44 AD 65.38
Best eROI2 = 6.23 t_index 0 thr_mc 0.7 thr_md 0.6 epoch 0 in 330 GSP 61.21 ADA 49.44 AD 65.38
Best eROI3 = 2.93 t_index 0 thr_mc 0.7 thr_md 0.6 epoch 0 in 330 GSP 61.21 ADA 49.44 AD 65.38
Best eROI4 = -0.37 t_index 0 thr_mc 0.7 thr_md 0.6 epoch 0 

In [ ]:
# For Tr/Cv dataset

from kaissandra.automate import combine_models
import numpy as np

rootname_config = 'RNN01011'
outputGain = .8
movingWindow = 500
nEventsPerStat = 5000
results_from = 'COMB'
feats_from_bids = False
save_journal = True
resolution = 20
thresholds_mc = [.5]#[int(np.round((.5+i/resolution)*100))/100 for i in range(int(resolution/2))]#[.7]#
thresholds_md = [.5]#[int(np.round((.5+i/resolution)*100))/100 for i in range(int(resolution/2))]#[.6]#
epochs = [14,14]#[9,11,6,8,6]
sufix='k1k2'+'E14E14'+'NYdec'
t_indexes = [0]
if feats_from_bids:
    ext = 'B'
else:
    ext = 'A'
first_day = '2016.01.01'#'2018.11.12'#
last_day = '2018.11.09'#'2019.03.29'#
list_index2sets = ['No','Cv']#['No','Cv']#['No','Cv','No']#
edge_dates = ['2017.9.27']#['2018.4.10']#['2017.9.27','2018.4.10']#
combine_ts = {'if_combine':False,'params_combine':[{'alg':'mean'}]}

entries = {'thresholds_mc':thresholds_mc,'thresholds_md':thresholds_md,'resolution':resolution,'results_from':results_from,
           'save_journal':save_journal,'feats_from_bids':feats_from_bids,'size_output_layer':8,'n_bits_outputs':[1,2,5],
           'combine_ts':combine_ts,'outputGain':outputGain,'movingWindow':movingWindow,
          'nEventsPerStat':nEventsPerStat,'first_day':first_day,'last_day':last_day,'list_index2sets':list_index2sets,'edge_dates':edge_dates,
          'build_test_db':False,'save_stats':False,'t_indexes':t_indexes,'combine_ts':combine_ts}

model_names = ['WRNN01011k1K5A','WRNN01011k2K5A']

tag_from_till = 'F170927T181109'#'F180410T181109'#'F170927T180410'#
combine_models(entries, model_names, epochs, rootname_config, sufix=sufix, melting_func='mean', tag_from_till=tag_from_till,log='')

In [ ]:
from kaissandra.results2 import merge_results

IDrs=['RRNN01010CMF180410T181109ACk1K5E12R100','RRNN01010CMF170927T180410ACk2K5E14R100']#['R'+rootname_config+tag+extR+sufix for fold_idx in range(k_end)]
IDr_merged='RRNN01010MEF170927T181109ACk1k2K5E12E14R100'
merge_results(IDrs, IDr_merged)

## Get Info from Candidate

In [ ]:
start_date = dt.date(2018, 11, 12)
start_date_monday = (start_date - dt.timedelta(days=start_date.weekday()))
end_date = dt.date(2019, 3, 29)
num_of_weeks = math.ceil((end_date - start_date_monday).days / 7.0)
print(num_of_weeks)

In [ ]:
from kaissandra.local_config import local_vars
from kaissandra.results2 import get_extended_results,remove_outliers

results_dir  = local_vars.results_directory
extfile = '.csv'

In [ ]:
rootname_config = 'RNN00010'
extR = 'AC'
sufix='R100'
K = 5
k_end = 2
k_init = 0
list_results_names=['R'+rootname_config+'k'+str(fold_idx+1)+'K'+str(K)+extR+sufix for fold_idx in range(k_init, k_end)]
list_epochs = [14 for _ in range(k_init, k_end)]
list_tis = ['mean' for _ in range(k_init, k_end)]
list_mgs = ['0.5','0.5','0.5']#for _ in range(k_end) 

In [ ]:
rootname_config = 'RNN00010'
extR_L = 'AL'
extR_S = 'BS'
sufix='R100'
K = 5
k_end = 2
k_init = 0
IDLs=['R'+rootname_config+'k'+str(fold_idx+1)+'K'+str(K)+extR_L+sufix for fold_idx in range(k_init, k_end)]
IDSs=['R'+rootname_config+'k'+str(fold_idx+1)+'K'+str(K)+extR_S+sufix for fold_idx in range(k_init, k_end)]
list_results_names = IDLs+IDSs
list_epochs = [14 for _ in range(k_init, k_end)]+[12 for _ in range(k_init, k_end)]
list_tis = ['2' for _ in range(k_init, k_end)]+['1' for _ in range(k_init, k_end)]
list_mgs = ['0.5','0.5']+['0.5','0.5']

In [ ]:
rootname_config = 'RNN01010'
extR = 'AC'
sufix='R20'
K = 5
k_end = 2
k_init = 0
list_results_names=['R'+rootname_config+'k'+str(fold_idx+1)+'K'+str(K)+extR+sufix for fold_idx in range(k_init, k_end)]
list_epochs = [11 for _ in range(k_init, k_end)]
list_tis = ['2' for _ in range(k_init, k_end)]
list_mcs = ['0.75' for _ in range(k_init, k_end)]#for _ in range(k_end) 
list_mds = ['0.6' for _ in range(k_init, k_end)]

In [ ]:
rootname_config = 'RNN01010'
extR_L = 'AL'
extR_S = 'BS'
sufix='R20'
K = 5
k_end = 2
k_init = 0
IDLs=['R'+rootname_config+'k'+str(fold_idx+1)+'K'+str(K)+extR_L+sufix for fold_idx in range(k_init, k_end)]
IDSs=['R'+rootname_config+'k'+str(fold_idx+1)+'K'+str(K)+extR_S+sufix for fold_idx in range(k_init, k_end)]
list_results_names = IDLs+IDSs
list_epochs = [9 for _ in range(k_init, k_end)]+[10 for _ in range(k_init, k_end)]
list_tis = ['2' for _ in range(k_init, k_end)]+['2' for _ in range(k_init, k_end)]
list_mcs = ['0.7' for _ in range(k_init, k_end)]+['0.6' for _ in range(k_init, k_end)]
list_mds = ['0.6' for _ in range(k_init, k_end)]+['0.6' for _ in range(k_init, k_end)]

In [ ]:
rootname_config = 'RNN01040'
extR_L = 'AL'
extR_S = 'BS'
sufix='R20'
K = 5
k_end = 2
k_init = 0
IDLs=['R'+rootname_config+'k'+str(fold_idx+1)+'K'+str(K)+extR_L+sufix for fold_idx in range(k_init, k_end)]
IDSs=['R'+rootname_config+'k'+str(fold_idx+1)+'K'+str(K)+extR_S+sufix for fold_idx in range(k_init, k_end)]
list_results_names = IDLs+IDSs
list_epochs = [8 for _ in range(k_init, k_end)]+[9 for _ in range(k_init, k_end)]
list_tis = ['2' for _ in range(k_init, k_end)]+['2' for _ in range(k_init, k_end)]
list_mcs = ['0.9' for _ in range(k_init, k_end)]+['0.9' for _ in range(k_init, k_end)]
list_mds = ['0.6' for _ in range(k_init, k_end)]+['0.6' for _ in range(k_init, k_end)]

In [ ]:
rootname_config = 'RNN01011'
extR_L = 'AL'
extR_S = 'BS'
sufix='R20bis'
K = 5
k_end = 2
k_init = 0
IDLs=['R'+rootname_config+'k'+str(fold_idx+1)+'K'+str(K)+extR_L+sufix for fold_idx in range(k_init, k_end)]
IDSs=['R'+rootname_config+'k'+str(fold_idx+1)+'K'+str(K)+extR_S+sufix for fold_idx in range(k_init, k_end)]
list_results_names = IDLs+IDSs
list_epochs = [12 for _ in range(k_init, k_end)]+[12 for _ in range(k_init, k_end)]
list_tis = ['1' for _ in range(k_init, k_end)]+['1' for _ in range(k_init, k_end)]
list_mcs = ['0.65' for _ in range(k_init, k_end)]+['0.5' for _ in range(k_init, k_end)]
list_mds = ['0.55' for _ in range(k_init, k_end)]+['0.65' for _ in range(k_init, k_end)]

In [ ]:
rootname_config = 'RNN01021'
extR = 'AL'
sufix='R20'
K = 5
k_end = 2
k_init = 0
list_results_names=['R'+rootname_config+'k'+str(fold_idx+1)+'K'+str(K)+extR+sufix for fold_idx in range(k_init, k_end)]
list_epochs = [9 for _ in range(k_init, k_end)]
list_tis = ['2' for _ in range(k_init, k_end)]
list_mcs = ['0.7' for _ in range(k_init, k_end)]#for _ in range(k_end) 
list_mds = ['0.7' for _ in range(k_init, k_end)]
#epoch            9.000000
#t_index          2.000000
#thr_mc           0.600000
#thr_md           0.700000

In [ ]:
rootname_config = 'RNN01010'
extR = 'AL'
epoch = 12
t_index = 'mean'
thr_mc = .7
thr_md = .6
K = 5
k_end = 1
k_init = 0
sufix='testk'+str(k_end)+'E'+str(epoch)+'TmeanC7D6'
list_results_names=['R'+rootname_config+'FE'+extR+sufix for fold_idx in range(k_init, k_end)]
list_epochs = [epoch for _ in range(k_init, k_end)]
list_tis = [t_index for _ in range(k_init, k_end)]
list_mcs = [str(thr_mc) for _ in range(k_init, k_end)]#for _ in range(k_end) 
list_mds = [str(thr_md) for _ in range(k_init, k_end)]

In [ ]:
rootname_config = 'RNN01010'
extR = 'BS'
epoch = 7
t_index = 'mean'
thr_mc = .8
thr_md = .5
K = 5
k_end = 1
k_init = 0
sufix='testk'+str(k_end)+'E'+str(epoch)+'TmeanC8D5'
list_results_names=['R'+rootname_config+'FE'+extR+sufix for fold_idx in range(k_init, k_end)]
list_epochs = [epoch for _ in range(k_init, k_end)]
list_tis = [t_index for _ in range(k_init, k_end)]
list_mcs = [str(thr_mc) for _ in range(k_init, k_end)]#for _ in range(k_end) 
list_mds = [str(thr_md) for _ in range(k_init, k_end)]

In [ ]:
rootname_config = 'RNN01010'
extRs = ['AC']
epoch = 0
t_index = '0'
thr_mc = .75
thr_md = .6
K = 5
k_end = 1
k_init = 0
sufix='R100'
tag_from_till='F181112T190329'
models_tag = 'k1k2k3k4k5K5'
epoch_tag = 'E9E11E6E8E6'
list_results_names=['R'+rootname_config+'CM'+tag_from_till+extR+models_tag+epoch_tag+sufix for extR in extRs]
list_epochs = [epoch for _ in extRs]
list_tis = [t_index for _ in extRs]
list_mcs = [str(thr_mc) for _ in extRs]#for _ in range(k_end) 
list_mds = [str(thr_md) for _ in extRs]

## Test Dataset

In [ ]:
rootname_config = 'RNN01011'
extRs = ['AC']
epoch = 0
t_index = '0'
thr_mc = .7
thr_md = .54
K = 5
k_end = 1
k_init = 0
sufix=''
tag_from_till='F181112T190329'
models_tag = 'k1k2'
epoch_tag = 'E14E14'
list_results_names=['R'+rootname_config+'CM'+tag_from_till+extR+models_tag+epoch_tag+sufix for extR in extRs]
list_epochs = [epoch for _ in extRs]
list_tis = [t_index for _ in extRs]
list_mcs = [str(thr_mc) for _ in extRs]#for _ in range(k_end) 
list_mds = [str(thr_md) for _ in extRs]

In [ ]:
rootname_config = 'RNN01010'
extRs = ['AC']
epoch = 0
t_index = '0'
thr_mc = .7
thr_md = .6
K = 5
k_end = 1
k_init = 0
sufix='R100'
tag_from_till='F181112T190329'
models_tag = 'k1k2'
epoch_tag = 'E12E14'
list_results_names=['R'+rootname_config+'CM'+tag_from_till+extR+models_tag+epoch_tag+sufix for extR in extRs]
list_epochs = [epoch for _ in extRs]
list_tis = [t_index for _ in extRs]
list_mcs = [str(thr_mc) for _ in extRs]#for _ in range(k_end) 
list_mds = [str(thr_md) for _ in extRs]

In [ ]:
# Combine 01010 and 01011
rootname_config = ['RNN01010','RNN01011']
extRs = ['AC','AC']
epoch = 0
t_index = '0'
thr_mc = [.7,.65]
thr_md = [.6,.6]
K = 5
k_end = 1
k_init = 0
sufix=['','']
tag_from_till='F181112T190329'
models_tag = 'k1k2'
epoch_tag = ['E12E14','E14E14']
list_results_names=['R'+rootname_config[i]+'CM'+tag_from_till+extR+models_tag+epoch_tag[i]+sufix[i] for i,extR in enumerate(extRs)]
list_epochs = [epoch for _ in extRs]
list_tis = [t_index for _ in extRs]
list_mcs = [str(thr_mc[i]) for i,_ in enumerate(extRs)]#for _ in range(k_end) 
list_mds = [str(thr_md[i]) for i,_ in enumerate(extRs)]

## Tr/Cv Dataset

In [ ]:
rootname_config = 'RNN01011'
extRs = ['AC','AC']
epoch = 0
t_index = '0'
thr_mc = [.7,.65]
thr_md = [.65,.6]
K = 5
k_end = 1
k_init = 0
sufix=''
tag_from_till = ['F180410T181109','F170927T180410']
models_tag = ['k1','k2']
epoch_tag = 'E14'
list_results_names=['R'+rootname_config+'CM'+tag_from_till[i]+extR+models_tag[i]+epoch_tag+sufix for i,extR in enumerate(extRs)]
list_epochs = [epoch for _ in extRs]
list_tis = [t_index for _ in extRs]
list_mcs = [str(thr_mc[i]) for i,_ in enumerate(extRs)]#for _ in range(k_end) 
list_mds = [str(thr_md[i]) for i,_ in enumerate(extRs)]

In [ ]:
rootname_config = 'RNN01010'
extRs = ['AC','AC']
epoch = 0
t_index = '0'
thr_mc = [.7,.7]
thr_md = [.6,.6]
K = 5
k_end = 1
k_init = 0
sufix=''
tag_from_till = ['F180410T181109','F170927T180410']
models_tag = ['k1','k2']
epoch_tag = ['E14','E14']
list_results_names=['R'+rootname_config+'CM'+tag_from_till[i]+extR+models_tag[i]+epoch_tag[i]+sufix for i,extR in enumerate(extRs)]
list_epochs = [epoch for _ in extRs]
list_tis = [t_index for _ in extRs]
list_mcs = [str(thr_mc[i]) for i,_ in enumerate(extRs)]#for _ in range(k_end) 
list_mds = [str(thr_md[i]) for i,_ in enumerate(extRs)]

In [ ]:
rootname_config = ['RNN01010','RNN01010','RNN01011','RNN01011']
extRs = ['AC','AC','AC','AC']
epoch = 0
t_index = '0'
thr_mc = [.7,.7,.7,.7]
thr_md = [.6,.6,.6,.6]
K = 5
k_end = 1
k_init = 0
sufix=['','','','']
tag_from_till = ['F180410T181109','F170927T180410','F180410T181109','F170927T180410']
models_tag = ['k1','k2','k1','k2']
epoch_tag = ['E12','E14','E14','E14']
list_results_names=['R'+rootname_config[i]+'CM'+tag_from_till[i]+extR+models_tag[i]+epoch_tag[i]+sufix[i] for i,extR in enumerate(extRs)]
list_epochs = [epoch for _ in extRs]
list_tis = [t_index for _ in extRs]
list_mcs = [str(thr_mc[i]) for i,_ in enumerate(extRs)]#for _ in range(k_end) 
list_mds = [str(thr_md[i]) for i,_ in enumerate(extRs)]

In [ ]:
# Combine 01010 k1 and k2
rootname_config = ['RNN01010']
extRs = ['AC']
epoch = 0
t_index = '0'
thr_mc = .7
thr_md = .6
K = 5
k_end = 1
k_init = 0
sufix=['']
tag_from_till='170927T181109'
models_tag = 'k1k2'
epoch_tag = ['E12E14']
list_results_names=['R'+rootname_config[i]+'CMF'+tag_from_till+extR+models_tag+epoch_tag[i]+sufix[i] for i,extR in enumerate(extRs)]
list_epochs = [epoch for _ in extRs]
list_tis = [t_index for _ in extRs]
list_mcs = [str(thr_mc) for _ in extRs]#for _ in range(k_end) 
list_mds = [str(thr_md) for _ in extRs]

In [ ]:
# Combine 01011 k1 and k2
rootname_config = ['RNN01011']
extRs = ['AC']
epoch = 0
t_index = '0'
thr_mc = .65
thr_md = .6
K = 5
k_end = 1
k_init = 0
sufix=['']
tag_from_till='170927T181109'
models_tag = 'k1k2'
epoch_tag = ['E14E14']
list_results_names=['R'+rootname_config[i]+'CMF'+tag_from_till+extR+models_tag+epoch_tag[i]+sufix[i] for i,extR in enumerate(extRs)]
list_epochs = [epoch for _ in extRs]
list_tis = [t_index for _ in extRs]
list_mcs = [str(thr_mc) for _ in extRs]#for _ in range(k_end) 
list_mds = [str(thr_md) for _ in extRs]

In [ ]:
# Combine 01001/01011 and k1/k2
rootname_config = ['RNN01010']+['RNN01011']
extRs = ['AC','AC']#['AC']+['AC']
epoch = 0
t_index = '0'
thr_mc = [.7,.65]
thr_md = [.6,.6]
K = 5
k_end = 1
k_init = 0
sufix=['','']
tag_from_till='170927T181109'
models_tag = 'k1k2'
epoch_tag = ['E12E14','E14E14']
list_results_names=['R'+rootname_config[i]+'CMF'+tag_from_till+extR+models_tag+epoch_tag[i]+sufix[i] for i,extR in enumerate(extRs)]
list_epochs = [epoch for _ in extRs]
list_tis = [t_index for _ in extRs]
list_mcs = [str(thr_mc[i]) for i,_ in enumerate(extRs)]#for _ in range(k_end) 
list_mds = [str(thr_md[i]) for i,_ in enumerate(extRs)]

In [ ]:
print(list_results_names)

In [ ]:
pip_limit = 0.05
    
#Journal = pd.DataFrame()
#for l,name in enumerate(list_results_names):
#    journal_filename = results_dir+name+'/journal/J_E'+str(list_epochs[l])+'TI'+list_tis[l]+'MG'+list_mgs[l]+ext
#    new_journal = pd.read_csv(journal_filename,sep='\t')
#    Journal = Journal.append(new_journal).sort_values(by=['Asset','DTi']).reset_index().drop(labels='level_0',axis=1)
#    print(journal_filename)

Journal = pd.DataFrame()
for l,name in enumerate(list_results_names):
    journal_filename = results_dir+name+'/journal/J_E'+str(list_epochs[l])+'TI'+str(list_tis[l])+'MC'+str(list_mcs[l])+'MD'+str(list_mds[l])+extfile
    new_journal = pd.read_csv(journal_filename,sep='\t')
    Journal = Journal.append(new_journal).sort_values(by=['Asset','DTi']).reset_index().drop(labels='level_0',axis=1)
    print(journal_filename)

# filter journal
#pip_limit = 0.02
#pos_under_2p = positions['espread']<pip_limit
print("shape")
print(Journal.shape[0])
print(journal_filename)
n_days = num_of_weeks*5+1
pos_dirname = '../../RNN/results/MERGED/positions/'
pos_filename = '_'.join([entry for entry in list_results_names])
positions_summary, log = get_extended_results(Journal,
                                    5,
                                    n_days,
                                   get_positions=True,pos_dirname=pos_dirname,
                                   pos_filename=pos_filename+'.csv')
#print(positions_summary)

positions = pd.read_csv(pos_dirname+pos_filename+'.csv',sep='\t')
#print(positions)
GROIS99, ROIS99, idx_sorted, low_arg_goi, high_arg_goi = remove_outliers(np.array(positions.GROI),
                                                                         np.array(positions.spread), thr=.99)

pos_under_2p = positions['espread']<pip_limit
#pos_under_thr_99 = positions['espread']<pip_limit
#GROIS99 = GROIS99[pos_under_2p]
#ROIS99 = ROIS99[pos_under_2p]
positions['DTo'] = positions["Do"] +" "+ positions["To"]
pos_under_thr = positions[pos_under_2p]#.sort_values(by=['DTo'])
per_under_2p = 100*sum(pos_under_2p)/positions.shape[0]
tgsr = 100*sum(positions['GROI']>0)/positions.shape[0]
gsr = 100*sum(pos_under_thr['GROI']>0)/sum(pos_under_2p)
tsr = 100*sum(positions['ROI']>0)/positions.shape[0]
sr = 100*sum(positions[pos_under_2p]['ROI']>0)/sum(pos_under_2p)
mean_spread = positions[pos_under_2p]['spread'].mean()
print("total mean GROI")
print(positions['GROI'].mean())
print("mean GROI of selected")
print(positions[pos_under_2p]['GROI'].mean())
print("mean_spread of selected")
print(mean_spread)
print("Number of pos under "+str(pip_limit))
print(positions[pos_under_2p].shape[0])
print("per under pip_limit")
print(per_under_2p)
print("total gross success rate")
print(tgsr)
print("gross success rate")
print(gsr)
print("total success rate")
print(tsr)
print("success rate")
print(sr)
print("GROI for positions under "+str(pip_limit))
print(positions[pos_under_2p]['GROI'].sum())
print("ROI for positions under "+str(pip_limit))
print(positions[pos_under_2p]['ROI'].sum())
print("positions['GROI'].sum()-pip_limit*positions['GROI'].shape[0]")
print(positions['GROI'].sum()-pip_limit*positions['GROI'].shape[0])
print("# Assets")
print(positions['Asset'][pos_under_2p].unique().shape[0])
pos_under_thr.to_csv(pos_dirname+pos_filename+str(100*pip_limit)+'pFilt.csv', index=False, sep='\t')
#pos_under_ALBS = pos_under_thr


In [ ]:
RRNN01010CMF180410T181109ACk1E9R20
RRNN01010CMF170927T180410ACk2K5E14

## Plot Results Candidate

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

print(pos_dirname+pos_filename+str(100*pip_limit)+'pFilt.csv')
plt.rcParams['figure.figsize'] = [10, 5]
plt.figure(0)
mx = int(np.ceil(max(pos_under_thr['GROI'])))
mn = int(np.floor(min(pos_under_thr['ROI'])))
bins = [i/20 for i in range(20*mn,20*mx)]
histG = plt.hist(pos_under_thr['GROI'], bins=bins)
histR = plt.hist(pos_under_thr['ROI'], bins=bins)
plt.grid()

#pos_under_thr.index = range(pos_under_thr.shape[0])
plt.figure(1)
plt.plot(range(pos_under_thr.shape[0]),pos_under_thr['GROI'].cumsum())
plt.plot(range(pos_under_thr.shape[0]),pos_under_thr['ROI'].cumsum())
plt.grid()

plt.figure(2)
#print(pos_under_thr['ROI'])
print(positions[pos_under_2p]['GROI'].shape)
cumG = plt.plot(positions['GROI'].cumsum())#, bins=bins
cumR = plt.plot(positions['ROI'].cumsum())
plt.plot(np.cumsum(positions.GROI[(positions.GROI>=low_arg_goi) & (positions.GROI<=high_arg_goi)]))
plt.plot(np.cumsum(positions.ROI[(positions.GROI>=low_arg_goi) & (positions.GROI<=high_arg_goi)]))
plt.grid()


#positions
#grouped = pos_format.groupby(['asset'])
weekly_group = pos_under_thr.groupby([pd.to_datetime(pos_under_thr['Di']).dt.strftime('%W')])['ROI']
weekly_group_G = pos_under_thr.groupby([pd.to_datetime(pos_under_thr['Di']).dt.strftime('%W')])['GROI']
asset_group = pos_under_thr.groupby(['Asset'])
weekly_sum = weekly_group.sum()
weekly_sum_G = weekly_group_G.sum()
weekly_count = weekly_group.count()
#weekly_sum.cumsum()
#print(asset_group['GROI'].sum())
#print(asset_group['ROI'].sum())
#print(asset_group['ROI'].count())
plt.figure(3)
plt.plot(weekly_sum_G.cumsum())
plt.plot(range(weekly_sum.shape[0]), weekly_sum.cumsum())
plt.grid()
#print(weekly_sum_G)
#print(weekly_sum)#.cumsum()
print(weekly_sum_G.cumsum()[-1])
print(weekly_sum.cumsum()[-1])
#print(weekly_count)
#print(weekly_sum.cumsum())
#print(weekly_count.shape)
#print(pos_under_thr.groupby([pd.to_datetime(pos_under_thr['Di']).dt.strftime('%W')])['GROI'].sum())
#print(weekly_count)
plt.figure(4)
plt.grid()
plt.hist(weekly_count, bins=range(0,max(weekly_count),5))

#for name, group in asset_group:
#    print(name)
#    print(group.to_string())
#pd.to_datetime(positions['Di']).dt.strftime('%W')
#pd.to_datetime(positions['Di'])#.groupby('Name').resample('W-Mon', on='Date').sum().reset_index().sort_values(by='Date')



list_dates = [dt.datetime.strptime(date, '%Y.%m.%d %H:%M:%S') for date in pos_under_thr['DTo']]
dates = matplotlib.dates.date2num(list_dates)
plt.figure(5)
plt.plot_date(list_dates, pos_under_thr['GROI'].cumsum(),fmt='-')
plt.plot_date(list_dates, pos_under_thr['ROI'].cumsum(),fmt='-')
#plt.xticks(np.arange(min(list_dates), max(list_dates), 1000000))
plt.gcf().autofmt_xdate()
plt.grid()

In [ ]:
ind=pos_under_thr[pos_under_thr['Di']=='2018.10.26'].index[-1]
print(pos_under_thr['Di'].loc[ind])
print('GROI@checkpoint')
print(pos_under_thr['GROI'].loc[:ind].cumsum().iloc[-1])
print('ROI@checkpoint')
print(pos_under_thr['ROI'].loc[:ind].cumsum().iloc[-1])


In [ ]:
import matplotlib.pyplot as plt
import matplotlib

print(pos_dirname+pos_filename+str(100*pip_limit)+'pFilt.csv')
plt.rcParams['figure.figsize'] = [10, 5]
plt.figure(0)
mx = int(np.ceil(max(pos_under_thr['GROI'])))
mn = int(np.floor(min(pos_under_thr['ROI'])))
bins = [i/20 for i in range(20*mn,20*mx)]
histG = plt.hist(pos_under_thr['GROI'], bins=bins)
histR = plt.hist(pos_under_thr['ROI'], bins=bins)
plt.grid()

#pos_under_thr.index = range(pos_under_thr.shape[0])
plt.figure(1)
plt.plot(range(pos_under_thr.shape[0]),pos_under_thr['GROI'].cumsum())
plt.plot(range(pos_under_thr.shape[0]),pos_under_thr['ROI'].cumsum())
#plt.plot(range(pos_under_03.shape[0]),pos_under_03['ROI'].cumsum())
plt.grid()

plt.figure(2)
#print(pos_under_thr['ROI'])
print(positions[pos_under_2p]['GROI'].shape)
cumG = plt.plot(positions['GROI'].cumsum())#, bins=bins
cumR = plt.plot(positions['ROI'].cumsum())
plt.plot(np.cumsum(positions.GROI[(positions.GROI>=low_arg_goi) & (positions.GROI<=high_arg_goi)]))
plt.plot(np.cumsum(positions.ROI[(positions.GROI>=low_arg_goi) & (positions.GROI<=high_arg_goi)]))
plt.grid()


#positions
#grouped = pos_format.groupby(['asset'])
weekly_group = pos_under_thr.groupby([pd.to_datetime(pos_under_thr['Di']).dt.strftime('%W')])['ROI']
weekly_group_G = pos_under_thr.groupby([pd.to_datetime(pos_under_thr['Di']).dt.strftime('%W')])['GROI']
asset_group = pos_under_thr.groupby(['Asset'])
weekly_sum = weekly_group.sum()
weekly_sum_G = weekly_group_G.sum()
weekly_count = weekly_group.count()
#weekly_sum.cumsum()
#print(asset_group['GROI'].sum())
#print(asset_group['ROI'].sum())
#print(asset_group['ROI'].count())
plt.figure(3)
plt.plot(weekly_sum_G.cumsum())
plt.plot(range(weekly_sum.shape[0]), weekly_sum.cumsum())
plt.grid()
#print(weekly_sum_G)
#print(weekly_sum)#.cumsum()
print(weekly_sum_G.cumsum()[-1])
print(weekly_sum.cumsum()[-1])
#print(weekly_count)
#print(weekly_sum.cumsum())
#print(weekly_count.shape)
#print(pos_under_thr.groupby([pd.to_datetime(pos_under_thr['Di']).dt.strftime('%W')])['GROI'].sum())
#print(weekly_count)
plt.figure(4)
plt.grid()
plt.hist(weekly_count, bins=range(0,max(weekly_count),5))

#for name, group in asset_group:
#    print(name)
#    print(group.to_string())
#pd.to_datetime(positions['Di']).dt.strftime('%W')
#pd.to_datetime(positions['Di'])#.groupby('Name').resample('W-Mon', on='Date').sum().reset_index().sort_values(by='Date')



list_dates = [dt.datetime.strptime(date, '%Y.%m.%d %H:%M:%S') for date in pos_under_thr['DTo']]
dates = matplotlib.dates.date2num(list_dates)
plt.figure(5)
plt.plot_date(list_dates, pos_under_thr['GROI'].cumsum(),fmt='-')
plt.plot_date(list_dates, pos_under_thr['ROI'].cumsum(),fmt='-')
#plt.plot_date([dt.datetime.strptime(date, '%Y.%m.%d %H:%M:%S') for date in pos_under_ALBS['DTo']],pos_under_ALBS['ROI'].cumsum(),fmt='*-')
#plt.xticks(np.arange(min(list_dates), max(list_dates), 1000000))
plt.gcf().autofmt_xdate()
plt.grid()

In [ ]:
pos_under_thr.sort_values(by=['ROI'],ascending=True).reset_index().drop('index',1)